# Playing with Coronavirus Timeseries

- https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset


## To Do:

- [ ] Add data from Covid Tracking Project's API
    - https://covidtracking.com/api
    
- [ ] Move app styling to a css file in a new `assets/` folder

### RESOURCES FOR FUTURE
- RAFAEL STUDY GROUP FOR MAKING A MAP
    - https://www.youtube.com/watch?v=MAhK7NHXEOg&feature=emb_logo
    - https://github.com/erdosn/additional-topic-plotly

In [1]:
import os,glob,sys
import re

In [2]:
!pip install -U fsds
from fsds.imports import *


Bad key "text.kerning_factor" on line 4 in
/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
fsds v0.2.15 loaded.  Read the docs: https://fs-ds.readthedocs.io/en/latest/ 


Handle,Package,Description
dp,IPython.display,Display modules with helpful display and clearing commands.
fs,fsds,Custom data science bootcamp student package
mpl,matplotlib,Matplotlib's base OOP module with formatting artists
plt,matplotlib.pyplot,Matplotlib's matlab-like plotting module
np,numpy,scientific computing with Python
pd,pandas,High performance data structures and tools
sns,seaborn,High-level data visualization library based on matplotlib


[i] Pandas .iplot() method activated.


```python
## Determine if dataset is downloaded via Kaggle CLI
import os
from zipfile import ZipFile


if DOWNLOAD:
    !pip install kaggle --upgrade

    cmd = 'kaggle datasets download -d sudalairajkumar/novel-corona-virus-2019-dataset'
    os.system(cmd)
    print('Downloaded dataset Zipfie, extracting...')
    with ZipFile('novel-corona-virus-2019-dataset.zip') as file:
        file.extractall()
```

# Function-izing Processing

1. `download_coronavirus_data(fpath='New Data/)` 
2. `1oad_raw_df(filepaths[4])`
3. `get_and_clean_US`
4. `set_datetime_index`

### Making a class to contain functions

### def  `download_coronavirus_data`

In [71]:
# @add_method(CoronaData)
def download_coronavirus_data(path='New Data/',verbose=False):
    """Installs the Kaggle Command Line Interface to clone dataset.
    Then extracts dataset to specified path and displays name of main file.
    Args:
        path(str): Folder to extract dataset into (must end with a '/')
        
    Returns:
        file_list(list): List of full filepaths to downloaded csv files.
    """
    ## Determine if dataset is downloaded via Kaggle CL
    import os,glob
    from zipfile import ZipFile
    from IPython.display import clear_output
    os.makedirs(path, exist_ok=True)

    ## Install Kaggle 
    !pip install kaggle --upgrade
    clear_output()
    
    ## Run Kaggle Command 
    cmd = 'kaggle datasets download -d sudalairajkumar/novel-corona-virus-2019-dataset'
    os.system(cmd)
    
    ## Extract ZipFile
    print(f'Downloaded dataset Zipfie, extracting to {path}...')
    zip_filepath = 'novel-corona-virus-2019-dataset.zip'
    with ZipFile(zip_filepath) as file:
        file.extractall(path)
    
    ## Delete Zip File
    os.system(f"rm {zip_filepath}"  )
    
    
    ## Get list of all csvs
    print('[i] Extraction Complete.')    
    file_list = glob.glob(path+"*.csv")
    
    
    ## Find main df 
    main_file = [file for file in file_list if 'covid_19_data.csv' in file]
    if verbose:
        print(f"[i] The main file name is {main_file}")
    return main_file[0] #file_list[index]

In [73]:
DOWNLOAD = True

if DOWNLOAD:
    main_file = download_coronavirus_data()
    print('\n\n[i] Success. Downloaded dataset from kaggle...')

else:
    print('Using pre-existing data')

Downloaded dataset Zipfie, extracting to New Data/...
[i] Extraction Complete.


[i] Success. Downloaded dataset from kaggle...


## Main Dataset - Get US States

### def `load_raw_df`

In [5]:
# @add_method(CoronaData)
def load_raw_df(fpath,kws={},verbose=True):
    """Performs most basic of preprocessing, including renaming date column to 
    Date and dropping 'Last Update', and 'SNo' columns"""
    
    ## Default Kws
    read_kws = dict(parse_dates=['ObservationDate','Last Update'])
    
    ## Add User kws
    read_kws = {**read_kws,**kws}
    
    if verbose:
        print(f"[i] Loading {fpath} with read_csv kws:",end='')
        display(read_kws)
        
    df = pd.read_csv(fpath,**read_kws)
    
    ## Drop unwated columns
    df.drop(['Last Update',
             'SNo'],axis=1,inplace=True)

    ## Rename Date columns
    df.rename({'ObservationDate':'Date'},axis=1,inplace=True)
    
    ## Display some info 
    if verbose:
        display(df.head())
        # Countries in the dataset
        print(f"[i] There are {len(df['Country/Region'].unique())} countries in the datatset")
        
        ## Get first and last date
        start_ts = df["Date"].loc[df['Date'].idxmin()].strftime('%m-%d-%Y')
        end_ts = df["Date"].loc[df['Date'].idxmax()].strftime('%m-%d-%Y')
        # DF['Date'].idxmin(), DF['Date'].idxmax()
        print(f"[i] Dates Covered:\n\tFrom {start_ts} to {end_ts}")

    return df

In [6]:
# raw_df = load_raw_df(main_file)

### def `set_datetime_index`

In [7]:
# @add_method(CoronaData)
def set_datetime_index(df_,col='Date'):#,drop_old=False):
    df = df_.copy()
    df[col] = pd.to_datetime(df[col],infer_datetime_format=True)
    df.set_index(df[col],drop=True,inplace=True)
    if col in df.columns:
        df.drop(columns=col,inplace=True)
    return df#, inplace=True)


### 06/27 Adding US Population

In [8]:
# state_url = 'https://raw.githubusercontent.com/jirvingphd/List-of-US-States/master/states.csv'
# state_lookup = pd.read_csv(state_url)

# state_lookup = state_lookup.append({'State':'Puerto Rico','Abbreviation':'PR'},ignore_index=True)

# state_lookup

In [9]:
# state_lookup.to_csv('Reference Data/united_states_abbreviations.csv', index=False)

In [10]:
# abbrev = pd.read_csv('Reference Data/united_states_abbreviations.csv')
# abbrev

In [11]:
# ## Making Master Lookup CSV
# abbrev = pd.read_csv('Reference Data/united_states_abbreviations.csv')
# display(abbrev.head())

In [12]:
# pop = pd.read_csv('Reference Data/us-pop-est2019-alldata.csv')
# pop.head()

In [13]:
# us_pop = pop[['NAME','POPESTIMATE2019']].copy() #.loc[pop['STATE']>0][
# us_pop.head()

In [14]:
# us_info = pd.merge(abbrev,us_pop,right_on='NAME',left_on='State',how="outer")
# # us_info.drop('NAME',axis=1,inplace=True)
# us_info

In [15]:
# us_info.to_csv('Reference Data/_US_state_info_merged.csv',index=False)

### def `load_us_reference_info`

In [16]:
# @add_method(CoronaData)
def load_us_reference_info():
    ## Making Master Lookup CSV
    abbrev = pd.read_csv('Reference Data/united_states_abbreviations.csv')
    pop = pd.read_csv('Reference Data/us-pop-est2019-alldata.csv')
    us_pop = pop.loc[pop['STATE']>0][['NAME','POPESTIMATE2019']].copy()
    us_info = pd.merge(abbrev,us_pop,right_on='NAME',left_on='State',how="inner")
    us_info.drop('NAME',axis=1,inplace=True)
    return us_info

### def  `calculate_per_capita`

In [17]:
# @add_method(CoronaData)
def calculate_per_capita(df_,stat_cols = ['Confirmed','Deaths','Recovered']):
    df = df_.copy()
    ## ADDING PER CAPITA DATA 
    for col in stat_cols:
        df[f"{col} Per Capita"] = df[col]/df['POPESTIMATE2019']
    df.drop('POPESTIMATE2019',axis=1,inplace=True)
    return df

### def `get_and_clean_US`

In [18]:
# @add_method(CoronaData)
def get_and_clean_US(df,save_as = 'Reference Data/united_states_abbreviations.csv',
                     make_date_index=True,per_capita=True):
    """Takes raw df loaded and extracts United States and processes
    all state names to create new abbreviation column 'state'.
    """
    ## Get only US
    df_us = df.groupby('Country/Region').get_group('US').copy() 
    
    # REPLACE STATE NAMES
    # https://raw.githubusercontent.com/jirvingphd/List-of-US-States/master/states.csv
#     try:
#         state_url = 'https://raw.githubusercontent.com/jirvingphd/List-of-US-States/master/states.csv'
#         state_lookup = pd.read_csv(state_url)
#         state_lookup.to_csv(save_as, index=False)
#     except:
#         print('Error downloading state lookup csv. Using local copy')
#         state_lookup.read_csv(save_as)
    state_lookup = load_us_reference_info()

    
    ## Make renaming dict for states
    STATE_DICT = dict(zip(state_lookup['State'],state_lookup['Abbreviation']))
    STATE_DICT.update({'Chicago':'IL',
                      'Puerto Rico':'PR',
                      'Virgin Islands':'VI',
                      'United States Virgin Islands':'VI'})

    ## Separately Process Rows that contain a city, state 
    df_city_states = df_us[df_us['Province/State'].str.contains(',')]
    
    
    ## Finding City Abbreviations in city_states
    import re
    state_expr = re.compile(r"[A-Z\.]{2,4}")
    df_city_states['state'] = df_city_states['Province/State'].apply(state_expr.findall)
    df_city_states = df_city_states.explode('state')


    ## Seperately process Rows that do not contain a city,state
    df_states = df_us[~df_us['Province/State'].str.contains(',')]
    df_states['state'] =  df_states['Province/State'].map(STATE_DICT)

    ## Combining data frame back together
    df = pd.concat([df_states,df_city_states]).sort_index()
    df = df.dropna(subset=['state'])
    
    ## Fix some stragglers (like D.C. vs DC)
    df['state'] = df['state'].replace('D.C.','DC')
    
    ## Add Population Data
    df = pd.merge(df, state_lookup,left_on='state',right_on="Abbreviation")
    
    if per_capita:

        for col in  ['Confirmed','Deaths','Recovered']:
            df[f"{col} Per Capita"] = df[col]/df['POPESTIMATE2019']
            
            ## Remove Population 
        df.drop('POPESTIMATE2019',axis=1,inplace=True)
#     if len(save_as)>0:
#         print(f'[i] Saving final df as {save_as}')
#         df.to_csv(save_as,index=False)
        
    if make_date_index:
        df = set_datetime_index(df)
    return df

In [19]:
state_lookup = load_us_reference_info()
state_lookup.head()

,State,Abbreviation,POPESTIMATE2019
0,Alabama,AL,4903185
1,Alaska,AK,731545
2,Arizona,AZ,7278717
3,Arkansas,AR,3017804
4,California,CA,39512223


### `def get_state_ts`

In [20]:
def get_state_ts(df, state_name,
                     group_col='state', ts_col=None,
                     freq='D', agg_func='sum'):
    """Take df_us and extracts state's data as then Freq/Aggregation provided"""
    
    ## Get state_df group
    state_df = df.groupby(group_col).get_group(state_name)#.resample(freq).agg(agg)
    
    ## Resample and aggregate state data
    state_df = state_df.resample(freq).agg(agg_func)
    
    
    ## Get and Rename Sum Cols 
    orig_cols = state_df.columns

    ## Create Renamed Sum columns
    for col in orig_cols:
        state_df[f"{state_name} - {col}"] = state_df[col]
      
    ## Drop original cols
    state_df.drop(orig_cols,axis=1,inplace=True)
    
    if ts_col is not None:
        ts_cols_selected = [col for col in state_df.columns if ts_col in col]
        state_df = state_df[ts_cols_selected]

    return state_df

In [21]:
# df = get_and_clean_US(raw_df)
# df

### Testing workflow so far

In [22]:
main_file = download_coronavirus_data()
raw_df = load_raw_df(main_file)
df = get_and_clean_US(raw_df)
display(df.head()) 
NY = get_state_ts(df,'NY')
NY.head()

Downloaded dataset Zipfie, extracting to New Data/...
[i] Extraction Complete.
[i] Loading New Data/covid_19_data.csv with read_csv kws:

{'parse_dates': ['ObservationDate', 'Last Update']}

,Date,Province/State,Country/Region,Confirmed,Deaths,Recovered
0,2020-01-22,Anhui,Mainland China,1.0,0.0,0.0
1,2020-01-22,Beijing,Mainland China,14.0,0.0,0.0
2,2020-01-22,Chongqing,Mainland China,6.0,0.0,0.0
3,2020-01-22,Fujian,Mainland China,1.0,0.0,0.0
4,2020-01-22,Gansu,Mainland China,0.0,0.0,0.0


[i] There are 223 countries in the datatset
[i] Dates Covered:
	From 01-22-2020 to 06-22-2020


,Province/State,Country/Region,Confirmed,Deaths,Recovered,state,State,Abbreviation,Confirmed Per Capita,Deaths Per Capita,Recovered Per Capita
Date,,,,,,,,,,,
2020-01-22,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.0,0.0
2020-01-23,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.0,0.0
2020-01-24,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.0,0.0
2020-01-25,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.0,0.0
2020-01-26,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.0,0.0


,NY - Confirmed,NY - Deaths,NY - Recovered,NY - Confirmed Per Capita,NY - Deaths Per Capita,NY - Recovered Per Capita
Date,,,,,,
2020-03-02,1.0,0.0,0.0,5.140447e-08,0.0,0.0
2020-03-03,2.0,0.0,0.0,1.028089e-07,0.0,0.0
2020-03-04,11.0,0.0,0.0,5.654492e-07,0.0,0.0
2020-03-05,23.0,0.0,0.0,1.182303e-06,0.0,0.0
2020-03-06,36.0,0.0,0.0,1.850561e-06,0.0,0.0


# class `CoronaData`

In [23]:
def add_method(cls):
    from functools import wraps # This convenience func preserves name and docstring

    """source=https://medium.com/@mgarod/dynamically-add-a-method-to-a-class-in-python-c49204b85bd6"""
    def decorator(func):
        @wraps(func) 
        def wrapper(self, *args, **kwargs): 
            return func(*args, **kwargs)
        setattr(cls, func.__name__, wrapper)
        # Note we are not binding func, but wrapper which accepts self but does exactly the same as func
        return func # returning func means func can still be used normally
    return decorator


In [74]:
class CoronaData(object):

    def __init__(self,data_dir='New Data/',run_workflow=True,
                 download=False,load_data=True,verbose=True):
        self.__download = download
        self.__verbose = verbose
        self._data_folder = data_dir
        
        ## Download data or set local filepath
        if download:
            self.download_coronavirus_data(verbose=verbose)
        else:
            self.get_data_fpath(data_dir)
        
        ## Load df_raw and df
        if load_data:
            self.load_raw_df(verbose=verbose)
            
        if run_workflow:
            self.get_and_clean_US()
            self._make_state_dict()
            print('Full Worfklow Complete.')
            

    # @add_method(CoronaData)
    def download_coronavirus_data(self,path=None,verbose=None):
        """Installs the Kaggle Command Line Interface to clone dataset.
        Then extracts dataset to specified path and displays name of main file.
        Args:
            path(str): Folder to extract dataset into (must end with a '/')

        Returns:
            file_list(list): List of full filepaths to downloaded csv files.
        """        
        if verbose==None:
            verbose = self.__verbose
        print('[i] DOWNLOADING DATA USING KAGGLE API')
        if path is None:
            path = self._data_folder
                                  
        ## Determine if dataset is downloaded via Kaggle CL
        import os,glob
        from zipfile import ZipFile
        from IPython.display import clear_output
        os.makedirs(path, exist_ok=True)

        try:
            import kaggle
        except:
            ## Install Kaggle 
            !pip install kaggle --upgrade
            clear_output()
            if verbose: print('- Installed kaggle command line tool.')

        ## Run Kaggle Command 
        cmd = 'kaggle datasets download -d sudalairajkumar/novel-corona-virus-2019-dataset'
        os.system(cmd)

        ## Extract ZipFile
        print(f'Downloaded dataset Zipfie, extracting to {path}...')
        zip_filepath = 'novel-corona-virus-2019-dataset.zip'
        with ZipFile(zip_filepath) as file:
            file.extractall(path)
        
        ## Delete Zip File
        os.system(f"rm {zip_filepath}"  )
            
        self.get_data_fpath(path)

        
    def get_data_fpath(self,path):
        """save self._file_list and self._main_file"""
        import glob
        verbose = self.__verbose
        ## Get list of all csvs
        if verbose: print('[i] Extraction Complete.')    
        file_list = glob.glob(path+"*.csv")

        ## Find main df 
        main_file = [file for file in file_list if 'covid_19_data.csv' in file]
        if verbose: print(f"[i] The main file name is {main_file}")
            
        self._file_list = file_list
        self._main_file = main_file[0]
    
    
    
    def load_raw_df(self,fpath=None,kws={},verbose=True):
        """Performs most basic of preprocessing, including renaming date column to 
        Date and dropping 'Last Update', and 'SNo' columns"""
        import pandas as pd
        if fpath is None:
            fpath = self._main_file

        ## Default Kws
        read_kws = dict(parse_dates=['ObservationDate','Last Update'])

        ## Add User kws
        read_kws = {**read_kws,**kws}

        if verbose:
            print(f"[i] Loading {fpath} with read_csv kws:",end='')
            display(read_kws)

        df = pd.read_csv(fpath,**read_kws)
        
        self.df_raw = df.copy()
        ## Drop unwated columns
        df.drop(['Last Update',
                 'SNo'],axis=1,inplace=True)
        

        ## Rename Date columns
        df.rename({'ObservationDate':'Date'},axis=1,inplace=True)

        ## Display some info 
        if verbose:
            display(df.head())
            # Countries in the dataset
            print(f"[i] There are "+str(len(df['Country/Region'].unique()))+" countries in the datatset")

            ## Get first and last date
            start_ts = df["Date"].loc[df['Date'].idxmin()].strftime('%m-%d-%Y')
            end_ts = df["Date"].loc[df['Date'].idxmax()].strftime('%m-%d-%Y')
            # DF['Date'].idxmin(), DF['Date'].idxmax()
            print(f"[i] Dates Covered:\n\tFrom {start_ts} to {end_ts}")
                  

        self.df = df
        
    
    def set_datetime_index(self,df_=None,col='Date'):#,drop_old=False):
        """Returns df with specified column as datetime index"""
        import pandas as pd

        if df_ is None:
            df_ = self.df
            
        df = df_.copy()
        df[col] = pd.to_datetime(df[col],infer_datetime_format=True)
        df.set_index(df[col],drop=True,inplace=True)
        if col in df.columns:
            df.drop(columns=col,inplace=True)
        return df#, inplace=True)
    
    
    
    def load_us_reference_info(self):
        """Return and save US Reference Data"""
        ## Making Master Lookup CSV
        import pandas as pd
        abbrev = pd.read_csv('Reference Data/united_states_abbreviations.csv')
        pop = pd.read_csv('Reference Data/us-pop-est2019-alldata.csv')
        us_pop = pop.loc[pop['STATE']>0][['NAME','POPESTIMATE2019']].copy()
        us_info = pd.merge(abbrev,us_pop,right_on='NAME',left_on='State',how="inner")
        us_info.drop('NAME',axis=1,inplace=True)
        self.reference_data = us_info
        return us_info
    
    
    def calculate_per_capita(self,df_=None,stat_cols = ['Confirmed','Deaths','Recovered']):
        """Calculate Per Capita columns"""
        if df_ is None:
            df_ = self.df
            
        df = df_.copy()
        
        if 'POPESTIMATE2019' in df.columns==False:
            self.load_us_reference_info()
            
        ## ADDING PER CAPITA DATA 
        for col in stat_cols:
            df[f"{col} Per Capita"] = df[col]/df['POPESTIMATE2019']
        df.drop('POPESTIMATE2019',axis=1,inplace=True)
        return df    

    
    def get_and_clean_US(self,df=None,#save_as = 'Reference Data/united_states_abbreviations.csv',
                         make_date_index=True,per_capita=True):
        """Takes raw df loaded and extracts United States and processes
        all state names to create new abbreviation column 'state'.
        """
        import pandas as pd
        if df is None:
            df= self.df
            
        ## Get only US
        df_us = df.groupby('Country/Region').get_group('US').copy() 


        state_lookup = self.load_us_reference_info()


        ## Make renaming dict for states
        STATE_DICT = dict(zip(state_lookup['State'],state_lookup['Abbreviation']))
        STATE_DICT.update({'Chicago':'IL',
                          'Puerto Rico':'PR',
                          'Virgin Islands':'VI',
                          'United States Virgin Islands':'VI'})

        ## Separately Process Rows that contain a city, state 
        df_city_states = df_us[df_us['Province/State'].str.contains(',')]


        ## Finding City Abbreviations in city_states
        import re
        state_expr = re.compile(r"[A-Z\.]{2,4}")
        df_city_states['state'] = df_city_states['Province/State'].apply(state_expr.findall)
        df_city_states = df_city_states.explode('state')


        ## Seperately process Rows that do not contain a city,state
        df_states = df_us[~df_us['Province/State'].str.contains(',')]
        df_states['state'] =  df_states['Province/State'].map(STATE_DICT)

        ## Combining data frame back together
        df = pd.concat([df_states,df_city_states]).sort_index()
        df = df.dropna(subset=['state'])

        ## Fix some stragglers (like D.C. vs DC)
        df['state'] = df['state'].replace('D.C.','DC')

        
        ## Combine Cleaned Data 
        df = pd.merge(df, state_lookup,left_on='state',right_on="Abbreviation")
       
    
        ## Add Population Data
        if per_capita:

            for col in  ['Confirmed','Deaths','Recovered']:
                df[f"{col} Per Capita"] = df[col]/df['POPESTIMATE2019']

                ## Remove Population 
            df.drop('POPESTIMATE2019',axis=1,inplace=True)
    #     if len(save_as)>0:
    #         print(f'[i] Saving final df as {save_as}')
    #         df.to_csv(save_as,index=False)

        if make_date_index:
            df = self.set_datetime_index(df)
        
        self.df_us = df.copy()
        return df
    
    
    def get_state_ts(self,state_name,df=None,
                     group_col='state', ts_col=None,
                     freq='D', agg_func='sum'):
        """Take df_us and extracts state's data as then Freq/Aggregation provided"""
        ## 
        if df is None:
            df = self.df_us
            
            
        ## Get state_df group
        state_df = df.groupby(group_col).get_group(state_name)#.resample(freq).agg(agg)

        ## Resample and aggregate state data
        state_df = state_df.resample(freq).agg(agg_func)


        ## Get and Rename Sum Cols 
        orig_cols = state_df.columns

        ## Create Renamed Sum columns
        for col in orig_cols:
            state_df[f"{state_name} - {col}"] = state_df[col]

        ## Drop original cols
        state_df.drop(orig_cols,axis=1,inplace=True)

        if ts_col is not None:
            ts_cols_selected = [col for col in state_df.columns if ts_col in col]
            state_df = state_df[ts_cols_selected]

        return state_df 

    
    def _make_state_dict(self,df=None,col='state'):
        if df is None:
            df = self.df_us
        elif col not in df.columns:
            msg = f"{col} not in df.columns.\nColumns include:"+'\n'.join(df.columns)
            raise Exception(msg)
        state_list=df['state'].unique()

        STATES = {}
        for state in state_list:
            STATES[state] = self.get_state_ts(state)
        self.STATES = STATES

        
    ### CLASS DISPLAY RELATED ITEMS
    def _self_report(self,private=False,methods=True,attributes=True):
        import inspect
        attr_list = inspect.getmembers(self)
        dashes='---'*20
        report = [dashes]
        report.append("[i] CoronaData Contents:\n"+dashes)


        method_list=["METHODS:"]
        attribute_list=["ATTRIBUTES"]
        workflow_list = ["WORKFLOW:"]
        
        if private==False:
            startswithcheck = '_'
        else:
            startswithcheck ='__'
        
        ## Loop through all attr
        for item in attr_list:
            item_name = item[0]
            
            ## Exclude Private/Special Attrs
            if item_name.startswith(startswithcheck)== False:
                
                ## Get tf if item is method
                method_check = inspect.ismethod(item[1])
                
                ## If item is a method:
                if method_check==True:
                    method_list.append(item_name)
                ## If item is an attribute
                else: 
                    attribute_list.append(item_name) 
                    
                    
        ## Get workflow
        workflow_funcs = [self.download_coronavirus_data,
                         self.load_raw_df, self.get_and_clean_US]
        for i,method in enumerate(workflow_funcs):
            workflow_list.append(f"{i+1}. {method.__name__}")
            
#         [workflow_list.append(method.__name__) for method in workflow_funcs]
#         print(workflow)
#         workflow_list.extend(workflow_funcs)

        report.append('\n\t'.join(workflow_list))
        
        
        if methods:
            report.append('\n\t'.join(method_list))
        if attributes:
            report.append('\n\t'.join(attribute_list))
            

        
        return '\n'.join(report)
    
    
    

                
    def __repr__(self):
#         display(help(self))
    
        return self._self_report()#('\n'.join(self._method_report()))
        

In [75]:
corona = CoronaData(verbose=False)
corona
# corona.download_coronavirus_data()

Full Worfklow Complete.


------------------------------------------------------------
[i] CoronaData Contents:
------------------------------------------------------------
WORKFLOW:
	1. download_coronavirus_data
	2. load_raw_df
	3. get_and_clean_US
METHODS:
	calculate_per_capita
	download_coronavirus_data
	get_and_clean_US
	get_data_fpath
	get_state_ts
	load_raw_df
	load_us_reference_info
	set_datetime_index
ATTRIBUTES
	STATES
	df
	df_raw
	df_us
	reference_data

In [76]:
corona.df_us

,Province/State,Country/Region,Confirmed,Deaths,Recovered,state,State,Abbreviation,Confirmed Per Capita,Deaths Per Capita,Recovered Per Capita
Date,,,,,,,,,,,
2020-01-22,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
2020-01-23,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
2020-01-24,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
2020-01-25,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
2020-01-26,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2020-06-18,Puerto Rico,US,6111.0,147.0,0.0,PR,Puerto Rico,PR,1.913458e-03,0.000046,0.0
2020-06-19,Puerto Rico,US,6195.0,147.0,0.0,PR,Puerto Rico,PR,1.939760e-03,0.000046,0.0
2020-06-20,Puerto Rico,US,6463.0,147.0,0.0,PR,Puerto Rico,PR,2.023675e-03,0.000046,0.0


## Counts by State

In [30]:
## Report Total Cases
total_cases = df.groupby('state').sum()
total_cases.sort_values('Confirmed',0,0).head(20).style.bar(['Deaths','Confirmed'])

,Confirmed,Deaths,Recovered,Confirmed Per Capita,Deaths Per Capita,Recovered Per Capita
state,,,,,,
NY,25810938.000000,1936865.000000,0.000000,1.326798,0.099564,0.000000
NJ,10396573.000000,687210.000000,2.000000,1.170497,0.077369,0.000000
CA,6586585.000000,237451.000000,76.000000,0.166697,0.006010,0.000002
IL,6518049.000000,292433.000000,76.000000,0.514374,0.023077,0.000006
MA,5892246.000000,378377.000000,19.000000,0.854878,0.054897,0.000003
PA,4636803.000000,297564.000000,0.000000,0.362194,0.023244,0.000000
TX,3856818.000000,94515.000000,0.000000,0.133013,0.003260,0.000000
MI,3835706.000000,339305.000000,5.000000,0.384075,0.033975,0.000001
FL,3714191.000000,142199.000000,0.000000,0.172932,0.006621,0.000000


In [77]:
# df.groupby('state').resample('d').sum()

In [78]:
# ### Get daily stats by state
# df2= df.groupby('state').resample('d').sum()
# df2

In [79]:
# df.groupby(['state','Date']).resample("D").sum()

In [80]:
# df.index

In [35]:
# state_df = df.groupby('state').get_group('NY').resample('D').sum()
# state_df[[col for col in state_df.columns if 'Per Capita' in col]].plot()

In [81]:
# ## RUN FULL WORKFLOW
# main_file = download_coronavirus_data()
# raw_df = load_raw_df(main_file)
# df = get_and_clean_US(raw_df)
# df.head()
corona_data = CoronaData(run_workflow=True,verbose=False)
df = corona_data.df_us
df

Full Worfklow Complete.


,Province/State,Country/Region,Confirmed,Deaths,Recovered,state,State,Abbreviation,Confirmed Per Capita,Deaths Per Capita,Recovered Per Capita
Date,,,,,,,,,,,
2020-01-22,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
2020-01-23,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
2020-01-24,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
2020-01-25,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
2020-01-26,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2020-06-18,Puerto Rico,US,6111.0,147.0,0.0,PR,Puerto Rico,PR,1.913458e-03,0.000046,0.0
2020-06-19,Puerto Rico,US,6195.0,147.0,0.0,PR,Puerto Rico,PR,1.939760e-03,0.000046,0.0
2020-06-20,Puerto Rico,US,6463.0,147.0,0.0,PR,Puerto Rico,PR,2.023675e-03,0.000046,0.0


In [82]:
state_df = get_state_ts(df,"NY")
state_df

,NY - Confirmed,NY - Deaths,NY - Recovered,NY - Confirmed Per Capita,NY - Deaths Per Capita,NY - Recovered Per Capita
Date,,,,,,
2020-03-02,1.0,0.0,0.0,5.140447e-08,0.000000,0.0
2020-03-03,2.0,0.0,0.0,1.028089e-07,0.000000,0.0
2020-03-04,11.0,0.0,0.0,5.654492e-07,0.000000,0.0
2020-03-05,23.0,0.0,0.0,1.182303e-06,0.000000,0.0
2020-03-06,36.0,0.0,0.0,1.850561e-06,0.000000,0.0
...,...,...,...,...,...,...
2020-06-18,385760.0,30974.0,0.0,1.982979e-02,0.001592,0.0
2020-06-19,386556.0,31015.0,0.0,1.987071e-02,0.001594,0.0
2020-06-20,387272.0,31083.0,0.0,1.990751e-02,0.001598,0.0


#### Need to decide what form to leave df in for plotting functions

In [83]:
# def get_state_ts(df, state_name,
#                  group_col='state',
#                  ts_col=None,resample=False,agg='sum',dt_index='Date'):#, ts_col='Cases'):
#     """"Separates a state_name/group from the group_col of df
#     Returns a resampled aggregate timeseries"""
    
#     ts = df.groupby(group_col).get_group(state_name)
    
#     # ts.set_index('date',inplace=True)
#     ts = ts.reset_index(drop=False).set_index(dt_index)
    
#     if resample:
#         ts = ts.resample('d').agg(agg)
#         ts['state'] = state_name
        
#     ## Return specific columns if specificed
#     if ts_col is not None:
#         return ts[ts_col]#.rename(f"{ts_col.name}-{state_name}")
#     else:
#         return ts

In [84]:
    
# ts_NY = get_state_ts(df,'NY',
#                   ts_col=['Confirmed',"Deaths","Recovered"],resample=True)
# display(ts_NY.head())
# for col in['Confirmed','Deaths','Recovered']:
#     ts_NY[col+'-Cumulative'] = ts_NY[col].cumsum()
# NY = ts_NY.reset_index()
# NY

In [85]:
# import plotly.express as px
# px.line(NY,x='Day',y='Confirmed-Cumulative')
# px.line(NY,x='Day',y='Confirmed')

In [86]:
# get_state_ts(df,'NY',ts_col='Confirmed')#.plot()

### Get all states in a dict

In [87]:
state_list=df['state'].unique()
print(len(state_list))
state_list

52


array(['WA', 'IL', 'CA', 'AZ', 'MA', 'WI', 'TX', 'NE', 'OR', 'RI', 'NH',
       'FL', 'NY', 'GA', 'NC', 'NJ', 'NV', 'TN', 'MD', 'CO', 'PA', 'KY',
       'IN', 'MN', 'CT', 'SC', 'UT', 'HI', 'OK', 'VA', 'DC', 'KS', 'MO',
       'VT', 'IA', 'LA', 'OH', 'AK', 'AR', 'DE', 'ID', 'ME', 'MI', 'MS',
       'MT', 'NM', 'ND', 'SD', 'WV', 'WY', 'AL', 'PR'], dtype=object)

In [88]:
state_list=df['state'].unique()

STATES = {}
for state in state_list:
    STATES[state] = get_state_ts(df,state)

STATES.keys()

dict_keys(['WA', 'IL', 'CA', 'AZ', 'MA', 'WI', 'TX', 'NE', 'OR', 'RI', 'NH', 'FL', 'NY', 'GA', 'NC', 'NJ', 'NV', 'TN', 'MD', 'CO', 'PA', 'KY', 'IN', 'MN', 'CT', 'SC', 'UT', 'HI', 'OK', 'VA', 'DC', 'KS', 'MO', 'VT', 'IA', 'LA', 'OH', 'AK', 'AR', 'DE', 'ID', 'ME', 'MI', 'MS', 'MT', 'NM', 'ND', 'SD', 'WV', 'WY', 'AL', 'PR'])

In [89]:
# !pip install -U plotly
display(STATES['MD'].head())


,MD - Confirmed,MD - Deaths,MD - Recovered,MD - Confirmed Per Capita,MD - Deaths Per Capita,MD - Recovered Per Capita
Date,,,,,,
2020-03-06,3.0,0.0,0.0,4.962221e-07,0.0,0.0
2020-03-07,3.0,0.0,0.0,4.962221e-07,0.0,0.0
2020-03-08,5.0,0.0,0.0,8.270368e-07,0.0,0.0
2020-03-09,5.0,0.0,0.0,8.270368e-07,0.0,0.0
2020-03-10,8.0,0.0,0.0,1.323259e-06,0.0,0.0


### Plot selected states and columns

In [90]:
### Define states and columns to plot
plot_states = ['NY','MD','CA']
plot_cols = ['Confirmed']


## Make empty list for states to concat
concat_dfs = []

# Grab each state's df
for state in plot_states:
    dfs = STATES[state]
    
    ## for each plot_cols, find all columns that contain that col name
    for plot_col in plot_cols:
        concat_dfs.append(dfs[[col for col in dfs.columns if plot_col in col]])

        
## Concatenate final dfs
plot_df = pd.concat(concat_dfs,axis=1)#[STATES[s] for s in plot_states],axis=1).iplot()

## Plot concatenated dfs
pfig = plot_df.iplot(theme='solar',asFigure=True)
pfig

In [91]:
df

,Province/State,Country/Region,Confirmed,Deaths,Recovered,state,State,Abbreviation,Confirmed Per Capita,Deaths Per Capita,Recovered Per Capita
Date,,,,,,,,,,,
2020-01-22,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
2020-01-23,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
2020-01-24,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
2020-01-25,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
2020-01-26,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2020-06-18,Puerto Rico,US,6111.0,147.0,0.0,PR,Puerto Rico,PR,1.913458e-03,0.000046,0.0
2020-06-19,Puerto Rico,US,6195.0,147.0,0.0,PR,Puerto Rico,PR,1.939760e-03,0.000046,0.0
2020-06-20,Puerto Rico,US,6463.0,147.0,0.0,PR,Puerto Rico,PR,2.023675e-03,0.000046,0.0


# DASHBOARD FUNCTIONS & APP [06/28/20]

**DASHBOARD FEATURES**
- Current Features:
    - Plotly Time Series Plot for US States
    
- TO DO Features:
    - Plotly Map of Cases by State/Zipcode

        - Time Series Plot by Zipcode

**FOR PLOTTING:**
- Need a plotly function that will allow for choices of data to display
- Plotting Function options:
    - Type of Cases: 
        - (confirmed, deaths,recovered)
        - Per Capita versions of above
    - Display New Cases or Cumulative Cases
    - 

### Dashboard Functions

### def `plot_states`

In [101]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go


def plot_states(df, state_list, plot_cols = ['Confirmed'],df_only=False,
                new_only=False,plot_scatter=True,show=False):
    """Plots the plot_cols for every state in state_list.
    Returns plotly figure
    New as of 06/21"""
    
    ## Get state dataframes
    concat_dfs = []  
    STATES = {}
    
    ## Get each state
    for state in state_list:

        # Grab each state's df and save to STATES
        dfs = get_state_ts(df,state)
        STATES[state] = dfs

        ## for each plot_cols, find all columns that contain that col name
        for plot_col in plot_cols:
            concat_dfs.append(dfs[[col for col in dfs.columns if col.endswith(plot_col)]])#plot_col in col]])

    ## Concatenate final dfs
    plot_df = pd.concat(concat_dfs,axis=1)#[STATES[s] for s in plot_states],axis=1).iplot()
    
    
    ## Set title and df if new_only
    if new_only:
        plot_df = plot_df.diff()
        title = "Coronavirus Cases by State - New Cases"
    else:
        title = 'Coronavirus Cases by State - Cumulative'
    
    ## Reset Indes
    plot_df.reset_index(inplace=True)
    
    ## Return Df or plot
    if df_only==False:

        if np.any(['per capita' in x.lower() for x in plot_cols]):
            value_name = "# of Cases - Per Capita"
        else:
            value_name='# of Cases'
        pfig_df_melt = plot_df.melt(id_vars=['Date'],var_name='State',
                                    value_name=value_name)
        
        if plot_scatter:
            plot_func = px.scatter
        else:
            plot_func = px.line
        # Plot concatenated dfs
        pfig = plot_func(pfig_df_melt,x='Date',y=value_name,color='State',
                      width=800,height=500,title=title)
        
#         pfig.update_xaxes(rangeslider_visible=True)

                # Add range slider
        pfig.update_layout(
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=7,
                             label="1week",
                             step="day",
                             stepmode="backward"),
                        dict(count=14,
                             label="2weeks",
                             step="day",
                             stepmode="backward"),
                        dict(count=1,
                             label="1m",
                             step="month",
                             stepmode="backward"),
                        dict(count=6,
                             label="6m",
                             step="month",
                             stepmode="backward"),

                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=True
                ),
                type="date"
            )
        )
        
        if show:
            pfig.show()
            
        return pfig
    else:
        return plot_df#.reset_index()
    

In [102]:
## Using Function
pfig = plot_states(df,['NY','MD','KY','CA','FL','MA','DC','VA'],
                  plot_cols=['Confirmed'],plot_scatter=False)#,new_only=True)#,df_only=True)
pfig

In [103]:
# ## Using Function
# pfig = plot_states(df,['NY','MD','KY','CA','FL','MA','DC','VA'],
#                   plot_cols=['Confirmed','Deaths'],df_only=True)
# pfig.melt(id_vars=['Date'],var_name='State',
#                                     value_name='# of Cases')
        # Plot concatenated dfs


# pfig_df = plot_states(df,['NY','MD','DC','CA','FL','TX'],
#                       plot_cols=['Confirmed'],
#                       df_only=True)
# # display(pfig_df.head(5)) 

# pfig_df.diff()

# pfig_df_melt = pfig_df.melt(id_vars=['Date'],var_name='State',value_name='# of Cases')
# pfig_df_melt

# pfig = px.scatter(pfig_df_melt,x='Date',y='# of Cases',color='State')
# pfig

# pfig_df = plot_states(df,['NY','MD','KY','CA','FL'],plot_cols=['Confirmed','Deaths'],
          
#                       df_only=True)
# pfig_df.head()

## Dashboard

### Load Dashboard Data

In [104]:
## RUN FULL WORKFLOW
corona_data = CoronaData(verbose=False,run_workflow=True)
df = corona_data.df_us.copy()
df.head()

Full Worfklow Complete.


,Province/State,Country/Region,Confirmed,Deaths,Recovered,state,State,Abbreviation,Confirmed Per Capita,Deaths Per Capita,Recovered Per Capita
Date,,,,,,,,,,,
2020-01-22,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.0,0.0
2020-01-23,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.0,0.0
2020-01-24,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.0,0.0
2020-01-25,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.0,0.0
2020-01-26,Washington,US,1.0,0.0,0.0,WA,Washington,WA,1.313216e-07,0.0,0.0


### JupyterDash app

In [119]:
try:
    from jupyter_dash import JupyterDash
except:
    %conda install -c conda-forge -c plotly jupyter-dash
    from jupyter_dash import JupyterDash

## IMPORTS
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go


import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


def make_options(menu_choices):
    """Returns list of dictionary with {'label':menu_choice,'value':menu_choice}"""
    options = []
    for choice in menu_choices:
        options.append({'label':choice,'value':choice})
    return options

## Make Plot Cols list for options
stat_cols = ['Confirmed','Deaths','Recovered']
plot_cols = []
for column in stat_cols:
    plot_cols.extend([col for col in df.columns if column in col])

new_options = [{'label':'New Cases Only','value':1},
{'label':'Cumulative Cases','value':0}]



# Build App
app = JupyterDash()

app.layout = html.Div([
    html.H1("Coronavirus Analysis"),
    html.Div([
    html.Div(id="menu",children=[        
            dcc.Dropdown(id='choose_states',multi=True,placeholder='Select States',
                        options= make_options(df['state'].sort_values().unique( )),
                        value=['MD','NY','TX','CA','AZ'],style={'width':'90%','display':'block'}),
            
            html.Div(id='case_type_menu',children=[
                dcc.Dropdown(id='choose_cases',multi=False,
                            placeholder='Select Case Type', 
                            options=make_options(plot_cols),
                            value='Confirmed', 
                            style={'width':'80%','display':'inline-block'}
                            ),
            
            
            dcc.RadioItems(id='choose_new',#multi=False,
                        options=new_options, value=0,#make_options(['True','False']),
                        style={'display':'inline-block'}
                        )], 
            style={'display':'inline-block','padding-top':'2em'})
        ],
        style={'border':"1px solid gray",
        "display":"block-inline","width":'90%'
        })
,
        
    dcc.Graph(id='graph')
    ], )],
        style={'width':'100%','display':'block','margin-left':'auto',
        "border":"2px solid black"})
    #"display":'block'})border":"2px solid blue",

@app.callback(Output('graph','figure'),[Input('choose_states','value'),
                                       Input('choose_cases','value'),
                                       Input('choose_new','value')])
def update_output_div(states,cases,new_only):
    if isinstance(states,list)==False:
        states = [states]
    if isinstance(cases,list)==False:
        cases = [cases]
#     if new_only=='True':
#         new_only=True
#     else:
#         new_only=False
    pfig = plot_states(df,states,plot_cols=cases,new_only=new_only)
    return pfig

app.run_server(mode='inline')

# PRIOR WORK - COUNTRY TIME SERIES

In [52]:
stop

NameError: name 'stop' is not defined

In [53]:
# ## Get the column names from the file name
# def clean_name(x,split_on='_19_'):
#     name = x.split(split_on)[-1].replace('.csv','')
#     return name

# col_names = list(map(clean_name,ts_files))
# col_names

In [54]:
# files_to_load = dict(zip(col_names,ts_files))
# files_to_load

In [55]:
# deaths_df = pd.read_csv(files_to_load['deaths'])
# deaths_df

In [56]:
# df_ts_us = deaths_df.groupby('Country/Region').get_group('US')
# df_ts_us

In [57]:
# df_US = pd.melt(df_ts_us,id_vars=['Province/State','Country/Region','Lat','Long'], var_name='Day', value_name='Deaths')
# df_US

In [58]:
TS = {}
for country in df.groupby('Country/Region').groups:
    ts_temp = df.groupby('Country/Region').get_group(country)
#     ts_temp.set_index('Day',inplace=True)
    TS[country] =ts_temp

In [59]:
def get_country_ts(df, country,group_col='Country/Region',ts_col='Cases'):
    ts = df.groupby(group_col).get_group(country )
    ts.set_index('date',inplace=True)

    if ts_col is not None:
        return ts[ts_col]
    else:
        return tsx
usa = get_country_ts(df,'US')
usa




KeyError: "None of ['date'] are in the columns"

In [60]:

usa.iloc[-10:]

NameError: name 'usa' is not defined

In [61]:
italy = get_country_ts(df,'Italy')
italy.plot()

KeyError: 'Italy'

In [62]:
def get_country_cum_ts(df, group='US'):
    US = df.groupby('Country/Region').get_group(group)
    US.set_index('date',inplace=True)
    ts_us = US.drop(columns=['Province/State','Day','Country/Region'])['Cases'].cumsum()#.plot()
    return ts_us

In [63]:
ts_it = get_country_cum_ts(df,'Italy')
ts_us = get_country_cum_ts(df,'US')
ts_ch = get_country_cum_ts(df,'China')
ts_ko = get_country_cum_ts(df,'Korea, South')
ts_it.plot(label='Italty')
ts_us.plot(label='US')
plt.legend()

KeyError: 'Italy'

In [64]:
ts_it.head()

NameError: name 'ts_it' is not defined

In [65]:

# ts_df = pd.concat([ts_us,ts_it,ts_ch,ts_ko],axis=1)
# ts_df.plot(subplots=True)

In [66]:
# US = df.groupby('Country/Region').get_group('US')
# US.set_index('date',inplace=True)
# ts_us = US.drop(columns=['Province/State','Day','Country/Region'])['Cases'].cumsum()#.plot()
# ts_us.plot()

In [67]:
# IT = df.groupby('Country/Region').get_group('Italy')
# IT.set_index('date',inplace=True)
# ts_it = IT.drop(columns=['Province/State','Day','Country/Region'])['Cases'].cumsum()#.plot()

In [68]:
# pd.concat

In [69]:
ts_it.plot(label='Italy')
ts_us.plot(label='US')
# ts_ch.plot(label='China')
plt.legend()

NameError: name 'ts_it' is not defined

In [70]:
fig, ax = plt.subplots()
for country in TS:
    TS[country]['Cases'].cumsum().plot(label=country,ax=ax)

ax.legend(bbox_to_anchor=(1.1, 1.05))


# plt.legend()

KeyError: 'Cases'